<a href="https://colab.research.google.com/github/artstreet18/-/blob/main/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F_%D0%BF%D0%BE%D0%BA%D1%83%D0%BF%D0%BE%D0%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sp
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import np_utils
from keras import regularizers
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import log_loss



In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Категоризация покупок/input/train.csv')


ParserError: ignored

Первая проблема, с которой мы сталкиваемся, в данных существует два варианта разделения дробных чисел: через точку и через процент. Поэтому обычное чтение csv не работает.

pandas.errors.ParserError: Error tokenizing data. C error: Expected 1 fields in line 8, saw 2

Есть несколько вариантов решения:


1.   Выкинуть все строки в которых возникает подобная проблема(чаще всего это в напитках или молоке)
2.   Отредактировать их отдельно в Excel и считывать другой формат


---

Так как данных не так много было решено выбрать второй вариант


In [ ]:
train = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Категоризация покупок/input/train.xls', sheet_name='train' )


train.fillna('', inplace=True)

In [ ]:
train.head(10)

,check_id,name,category,price,count
0,0,*3479755 ТRUF.Конф.кр.корп.гл.вк.шок180г,Чай и сладкое,49.0,2.0
1,0,3408392 ECONTA Мешки д/мусора 30л 30шт,Для дома,21.0,1.0
2,0,3260497 ЯШКИНО Рулет С ВАР.СГУЩ. 200г,Чай и сладкое,39.0,1.0
3,0,3300573 Пакет ПЯТЕРОЧКА 65х40см,Упаковка,4.0,1.0
4,0,3413607 ЗЕР/СЕЛ.Сухари с изюмом 250г,Чай и сладкое,35.0,1.0
5,0,3221388 ШАРЛ.Печенье вафел.рассыпч.225г,Чай и сладкое,38.0,1.0
6,0,"*97452 ПРОСТ.Кефир 3,2% 930г",Молочка,55.0,1.0
7,0,57575 MILFORD Зам.сахара доз. 650таб,Бакалея,119.0,1.0
8,0,29880 ПИСК.Ацидоб.2.2%сл.пюр-пак0.5л,Молочка,34.0,1.0
9,1,ШОКОЛАДНОЕ ЯЙЦО 20Г КИНДЕР СЮРПРИЗ ФЕРРЕ,Дети,49.0,1.0


Одной из основных проблем моделей с препроцессингом является то, что их очень сложно масштабировать, так как улучшая данные мы делаем менее гибкую модель. Поэтому в данном случае я попробую получить максимальный результат из сырых данных без дополнительных датасетов. При получении новых данных модель будет очень просто обучить на них, такую модель можно спокойно вклинивать в pipeline.

Заметим, что данные достаточно разнородные. Где-то можно выделить отдельные слова, где-то нет. Однако для нас не важен порядок: «молоко свеж.» и «свеж. молоко» для нас одно и то же, поэтому стандартный Word2vec нам скорее вредит.

Рассмотрим модели на двух вариантах TfidfVectorizer и CountVectorizer

---

P.S. Далее первый блок кода использует tfidf, а второй count 

In [ ]:
tfidf_chars = TfidfVectorizer(analyzer='char', ngram_range=(2,7), max_features=8192)
tfidf_words = TfidfVectorizer(ngram_range=(1,6), max_features=16384)
count_chars = CountVectorizer(analyzer='char', ngram_range=(2,7), max_features=8192)
count_words = CountVectorizer(ngram_range=(1,6), max_features=16384)

Для нас важны как отдельные слова в моделях, так и сочетания символов. Поэтому объединим оба варианта.

In [ ]:
X = sp.sparse.hstack((tfidf_chars.fit_transform(train["name"]), tfidf_words.fit_transform(train["name"])))

labeler = LabelEncoder()
y = labeler.fit_transform(train.category)

TypeError: ignored

Получаем ошибку, потому что у нас не все данные считываются как строка. Зададим строку в явном виде.

In [ ]:
train['name'] = train['name'].apply(str)

In [ ]:
tfidf_chars = TfidfVectorizer(analyzer='char', ngram_range=(2,7), max_features=8192 )
tfidf_words = TfidfVectorizer(ngram_range=(1,6), max_features=16384)
count_chars = CountVectorizer(analyzer='char', ngram_range=(2,7), max_features=8192)
count_words = CountVectorizer(ngram_range=(1,6), max_features=16384)

Меняем y на categorical, чтобы input_shape совпадал далее с выходом softmax

In [ ]:
X = sp.sparse.hstack((tfidf_chars.fit_transform(train["name"]), tfidf_words.fit_transform(train["name"])))

labeler = LabelEncoder()
y = labeler.fit_transform(train["category"])
y = to_categorical(y)

In [ ]:
Xc = sp.sparse.hstack((count_chars.fit_transform(train["name"]), count_words.fit_transform(train["name"])))

labeler = LabelEncoder()
yc = labeler.fit_transform(train["category"])
yc = to_categorical(yc)

In [ ]:
y

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

В выборке не так много даных, поэтому вместо деления 70/30 придется использовать 80/20. Решения с меньшим количеством тестовой выборки значительно зависят от случайности, поэтому 20% оптимально.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=9)

In [ ]:
Xc_train, Xc_test, yc_train, yc_test = train_test_split(Xc, yc, test_size=0.2, shuffle=True, random_state=9)

In [ ]:
 y_train.shape

(9577, 25)

Далее стандартная простая модель с выходом softmax для классификации. Dropout для уменьшения переобучения. Early Stopping позволяет получить достаточно хорошо обученную модель в момент ухудшения. Дли нигилирования эффекта случайности на каждом этапе выставляем терпение на 2(2 раза должно быть ухудшение) 

Оптимальные параметры для модели перебираются случайным образом.

В качестве слоев были использованы [2048, 512, 25], [1024, 512, 25], [1024, 256, 25]

Для активации [sigmoid, sigmoid, softmax], [relu, relu, softmax], [relu, sigmoid, softmax]

Dropout [0.25, 0.25], [0.25, 0.15], [0.2, 0.2], [0.15, 0.15], [0.2, 0.15], [0.2, 0.1]

оптимизатор был выбран Adam, как наиболее стабильный 

lr = [0.0001, 0.0002, 0.0003, 0.0005, 0.001, 0.002, 0.005]

epochs не имеет значения, используем достаточно большой, чтобы выхоить за счёт Early Stopping

batch_size подбирался исходя из lr, чтобы модель достаточно быстро отрабатывала(не более минуты на эпоху)


In [ ]:
def create_model():
    model = Sequential()
    model.add(Dense(1024, input_dim=X.shape[1], activation='sigmoid', ))
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='sigmoid'))
    model.add(Dropout(0.1))
    model.add(Dense(25, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr = 0.0002), metrics=['accuracy'])
    return model

epochs = 40
batch_size = 30

model = create_model()
hist = model.fit(X_train, y_train,
                 batch_size=batch_size,
                 validation_data=(X_test, y_test),
                 epochs=epochs,
                 callbacks=[EarlyStopping(patience=2, monitor='val_loss')])

Epoch 1/40
365/365 [==============================] - 19s 52ms/step - loss: 2.9384 - accuracy: 0.1328 - val_loss: 2.6775 - val_accuracy: 0.3442
Epoch 2/40
365/365 [==============================] - 19s 51ms/step - loss: 2.6152 - accuracy: 0.2795 - val_loss: 2.1826 - val_accuracy: 0.5261
Epoch 3/40
365/365 [==============================] - 19s 51ms/step - loss: 2.0966 - accuracy: 0.4984 - val_loss: 1.6632 - val_accuracy: 0.6335
Epoch 4/40
365/365 [==============================] - 19s 52ms/step - loss: 1.5744 - accuracy: 0.6434 - val_loss: 1.2789 - val_accuracy: 0.7139
Epoch 5/40
365/365 [==============================] - 19s 51ms/step - loss: 1.2380 - accuracy: 0.7228 - val_loss: 1.0342 - val_accuracy: 0.7651
Epoch 6/40
365/365 [==============================] - 19s 52ms/step - loss: 0.9833 - accuracy: 0.7723 - val_loss: 0.8737 - val_accuracy: 0.7947
Epoch 7/40
365/365 [==============================] - 19s 52ms/step - loss: 0.7938 - accuracy: 0.8133 - val_loss: 0.7668 - val_accuracy:

In [136]:
def create_model():
    model = Sequential()
    model.add(Dense(1024, input_dim=X.shape[1], activation='sigmoid', ))
    model.add(Dropout(0.25))
    model.add(Dense(256, activation='sigmoid'))
    model.add(Dropout(0.25))
    model.add(Dense(25, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr = 0.0002), metrics=['accuracy'])
    return model

epochs = 40
batch_size = 30

model = create_model()
hist = model.fit(Xc_train, yc_train,
                 batch_size=batch_size,
                 validation_data=(Xc_test, yc_test),
                 epochs=epochs,
                 callbacks=[EarlyStopping(patience=2, monitor='val_loss')])

Epoch 1/40
365/365 [==============================] - 19s 52ms/step - loss: 2.7296 - accuracy: 0.2292 - val_loss: 1.6010 - val_accuracy: 0.6569
Epoch 2/40
365/365 [==============================] - 19s 52ms/step - loss: 1.4668 - accuracy: 0.6600 - val_loss: 0.9866 - val_accuracy: 0.7669
Epoch 3/40
365/365 [==============================] - 19s 51ms/step - loss: 0.9249 - accuracy: 0.7840 - val_loss: 0.7553 - val_accuracy: 0.8075
Epoch 4/40
365/365 [==============================] - 19s 51ms/step - loss: 0.6520 - accuracy: 0.8435 - val_loss: 0.6321 - val_accuracy: 0.8389
Epoch 5/40
365/365 [==============================] - 19s 51ms/step - loss: 0.4943 - accuracy: 0.8832 - val_loss: 0.5501 - val_accuracy: 0.8550
Epoch 6/40
365/365 [==============================] - 19s 52ms/step - loss: 0.3994 - accuracy: 0.9058 - val_loss: 0.4990 - val_accuracy: 0.8685
Epoch 7/40
365/365 [==============================] - 19s 52ms/step - loss: 0.3213 - accuracy: 0.9275 - val_loss: 0.4742 - val_accuracy:

Практически на всех протестированных данных TdidfVectorize получает более хорошие значения.

Одним из лучших результатов без предварительной обработки данных был:

In [ ]:
score = log_loss(y_test, model.predict(X_test))
score

0.3790361030197505